In [1]:
import numpy as np
import pandas as pd
import math
import re

In [2]:
order_report_df = pd.read_excel("Company X - Order Report.xlsx")
pincode_zones_df = pd.read_excel("Company X - Pincode Zones.xlsx")
SKU_master_df = pd.read_excel("Company X - SKU Master.xlsx")
invoice_df = pd.read_excel("Courier Company - Invoice.xlsx")
rates_df = pd.read_excel("Courier Company - Rates.xlsx")

In [3]:
# order report dataframe
order_report_df

,ExternOrderNo,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0
...,...,...,...
395,2001806229,8904223818942,1.0
396,2001806229,8904223818850,1.0
397,2001806226,8904223818850,2.0
398,2001806210,8904223816214,1.0


In [4]:
# Rename the column name: ExternOrderNumber to Order_Id
order_report_df.rename(columns={"ExternOrderNo":"Order_ID"}, inplace = True)
order_report_df

,Order_ID,SKU,Order Qty
0,2001827036,8904223818706,1.0
1,2001827036,8904223819093,1.0
2,2001827036,8904223819109,1.0
3,2001827036,8904223818430,1.0
4,2001827036,8904223819277,1.0
...,...,...,...
395,2001806229,8904223818942,1.0
396,2001806229,8904223818850,1.0
397,2001806226,8904223818850,2.0
398,2001806210,8904223816214,1.0


In [5]:
SKU_master_df

,SKU,Weight (g)
0,8904223815682,210
1,8904223815859,165
2,8904223815866,113
3,8904223815873,65
4,8904223816214,120
...,...,...
61,8904223819505,210
62,8904223819499,210
63,8904223819512,210
64,8904223819543,300


In [6]:
# convert this weight g into kg
SKU_master_df["Total_Weight_X"] = SKU_master_df["Weight (g)"]/1000
SKU_master_df.drop("Weight (g)", axis = 1, inplace = True)

In [7]:
SKU_master_df

,SKU,Total_Weight_X
0,8904223815682,0.210
1,8904223815859,0.165
2,8904223815866,0.113
3,8904223815873,0.065
4,8904223816214,0.120
...,...,...
61,8904223819505,0.210
62,8904223819499,0.210
63,8904223819512,0.210
64,8904223819543,0.300


In [8]:
SKU_master_df["Total_Weight_slab"] = np.where(SKU_master_df["Total_Weight_X"]<0.501, 0.5,
                                              np.where(SKU_master_df["Total_Weight_X"]<1.01,1,
                                                      SKU_master_df["Total_Weight_X"]))

In [9]:
SKU_master_df

,SKU,Total_Weight_X,Total_Weight_slab
0,8904223815682,0.210,0.5
1,8904223815859,0.165,0.5
2,8904223815866,0.113,0.5
3,8904223815873,0.065,0.5
4,8904223816214,0.120,0.5
...,...,...,...
61,8904223819505,0.210,0.5
62,8904223819499,0.210,0.5
63,8904223819512,0.210,0.5
64,8904223819543,0.300,0.5


In [10]:
order_report_df1 = order_report_df.merge(SKU_master_df, on = "SKU", how = "inner")
order_report_df1

,Order_ID,SKU,Order Qty,Total_Weight_X,Total_Weight_slab
0,2001827036,8904223818706,1.0,0.127,0.5
1,2001827036,8904223819093,1.0,0.150,0.5
2,2001827036,8904223819109,1.0,0.100,0.5
3,2001827036,8904223818430,1.0,0.165,0.5
4,2001827036,8904223819277,1.0,0.350,0.5
...,...,...,...,...,...
396,2001806229,8904223818942,1.0,0.133,0.5
397,2001806229,8904223818850,1.0,0.240,0.5
398,2001806226,8904223818850,2.0,0.240,0.5
399,2001806210,8904223816214,1.0,0.120,0.5


In [11]:
# Weight slab charged by Courier Company (KG)

invoice_df["Courier_Charged_slab"] = np.where(invoice_df["Charged Weight"] < 0.501,0.5,
                                       np.where(invoice_df["Charged Weight"] < 1, 1,
                                               np.where(invoice_df["Charged Weight"] < 1.5, 1.5,
                                                       np.where(invoice_df["Charged Weight"] < 2, 2,
                                                               np.where(invoice_df["Charged Weight"] < 2.5,2.5,
                                                                       np.where(invoice_df["Charged Weight"] < 3, 3,
                                                                               invoice_df["Charged Weight"]))))))

In [12]:
invoice_df.rename(columns = {"Order ID":"Order_ID"}, inplace = True)
invoice_df.rename(columns = {"Charged Weight":"Total_Weight_Courier"}, inplace = True)
invoice_df

,AWB Code,Order_ID,Total_Weight_Courier,Warehouse Pincode,Customer Pincode,Zone,Type of Shipment,Billing Amount (Rs.),Courier_Charged_slab
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,1.5
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,1.5
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,3.0
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,1.5
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,0.5
...,...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2,1.0
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7,0.5
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4,0.5
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,33.0,0.5


In [13]:
pincode_zones_df

,Warehouse Pincode,Customer Pincode,Zone
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d
...,...,...,...
119,121003,325207,b
120,121003,303702,b
121,121003,313301,b
122,121003,173212,e


In [14]:
pincode_zones_df.rename(columns = {"Zone":"Zone_X"}, inplace = True) # renaming zone column

In [15]:
pincode_zones_df

,Warehouse Pincode,Customer Pincode,Zone_X
0,121003,507101,d
1,121003,486886,d
2,121003,532484,d
3,121003,143001,b
4,121003,515591,d
...,...,...,...
119,121003,325207,b
120,121003,303702,b
121,121003,313301,b
122,121003,173212,e


In [16]:
# Delivery Zone charged by Courier Company
invoice_df.rename(columns = {"Zone":"Zone"}, inplace = True)
invoice_df.rename(columns = {"Type of Shipment":"Type_of_Shipment"}, inplace = True)

In [17]:
invoice_df

,AWB Code,Order_ID,Total_Weight_Courier,Warehouse Pincode,Customer Pincode,Zone,Type_of_Shipment,Billing Amount (Rs.),Courier_Charged_slab
0,1091117222124,2001806232,1.30,121003,507101,d,Forward charges,135.0,1.5
1,1091117222194,2001806273,1.00,121003,486886,d,Forward charges,90.2,1.5
2,1091117222931,2001806408,2.50,121003,532484,d,Forward charges,224.6,3.0
3,1091117223244,2001806458,1.00,121003,143001,b,Forward charges,61.3,1.5
4,1091117229345,2001807012,0.15,121003,515591,d,Forward charges,45.4,0.5
...,...,...,...,...,...,...,...,...,...
119,1091118551656,2001812941,0.73,121003,325207,d,Forward charges,90.2,1.0
120,1091117614452,2001809383,0.50,121003,303702,d,Forward and RTO charges,86.7,0.5
121,1091120922803,2001820978,0.50,121003,313301,d,Forward charges,45.4,0.5
122,1091121844806,2001811475,0.50,121003,173212,b,Forward charges,33.0,0.5


In [18]:
invoice_df = invoice_df.merge(order_report_df1, on = "Order_ID", how = "inner")
invoice_df

,AWB Code,Order_ID,Total_Weight_Courier,Warehouse Pincode,Customer Pincode,Zone,Type_of_Shipment,Billing Amount (Rs.),Courier_Charged_slab,SKU,Order Qty,Total_Weight_X,Total_Weight_slab
0,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,1.5,8904223818645,6.0,0.137,0.5
1,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,1.5,8904223819147,2.0,0.240,0.5
2,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,1.5,8904223819017,1.0,0.115,0.5
3,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,1.5,8904223818706,1.0,0.127,0.5
4,1091117222194,2001806273,1.0,121003,486886,d,Forward charges,90.2,1.5,8904223818942,1.0,0.133,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,1091121844806,2001811475,0.5,121003,173212,b,Forward charges,33.0,0.5,8904223819031,1.0,0.112,0.5
397,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,0.5,8904223816214,1.0,0.120,0.5
398,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,0.5,8904223819499,1.0,0.210,0.5
399,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,0.5,8904223819505,1.0,0.210,0.5


In [19]:
rate_col = [col for col in rates_df.columns if col.startswith("fwd_") or col.startswith("rto_")]
rate_col

['fwd_a_fixed',
 'fwd_a_additional',
 'fwd_b_fixed',
 'fwd_b_additional',
 'fwd_c_fixed',
 'fwd_c_additional',
 'fwd_d_fixed',
 'fwd_d_additional',
 'fwd_e_fixed',
 'fwd_e_additional',
 'rto_a_fixed',
 'rto_a_additional',
 'rto_b_fixed',
 'rto_b_additional',
 'rto_c_fixed',
 'rto_c_additional',
 'rto_d_fixed',
 'rto_d_additional',
 'rto_e_fixed',
 'rto_e_additional']

In [20]:
## melt in long form
rates_long = rates_df. melt(value_vars = rate_col, var_name = "zone_type", value_name = "Rate")
rates_long

,zone_type,Rate
0,fwd_a_fixed,29.5
1,fwd_a_additional,23.6
2,fwd_b_fixed,33.0
3,fwd_b_additional,28.3
4,fwd_c_fixed,40.1
5,fwd_c_additional,38.9
6,fwd_d_fixed,45.4
7,fwd_d_additional,44.8
8,fwd_e_fixed,56.6
9,fwd_e_additional,55.5


In [21]:
# using re module to search for string patten in dataset
rates_long["Zone"] = rates_long["zone_type"].str.extract(r"_(a|b|c|d|e)")
rates_long["Shipment"] = rates_long["zone_type"].str.extract(r"(fwd_|rto_)")

In [22]:
# creating a column in which type of shipment is defined
rates_long["Type_of_Shipment"] = np.where(rates_long["Shipment"] == "fwd_","Forward charges" , "Forward and RTO charges")
rates_long.rename(columns = {"Zone":"Zone"})

,zone_type,Rate,Zone,Shipment,Type_of_Shipment
0,fwd_a_fixed,29.5,a,fwd_,Forward charges
1,fwd_a_additional,23.6,a,fwd_,Forward charges
2,fwd_b_fixed,33.0,b,fwd_,Forward charges
3,fwd_b_additional,28.3,b,fwd_,Forward charges
4,fwd_c_fixed,40.1,c,fwd_,Forward charges
5,fwd_c_additional,38.9,c,fwd_,Forward charges
6,fwd_d_fixed,45.4,d,fwd_,Forward charges
7,fwd_d_additional,44.8,d,fwd_,Forward charges
8,fwd_e_fixed,56.6,e,fwd_,Forward charges
9,fwd_e_additional,55.5,e,fwd_,Forward charges


In [23]:
# Joining columns to invoice_df
invoice_df = invoice_df.merge(pincode_zones_df, on =["Warehouse Pincode"])

In [24]:
invoice_df

,AWB Code,Order_ID,Total_Weight_Courier,Warehouse Pincode,Customer Pincode_x,Zone,Type_of_Shipment,Billing Amount (Rs.),Courier_Charged_slab,SKU,Order Qty,Total_Weight_X,Total_Weight_slab,Customer Pincode_y,Zone_X
0,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,1.5,8904223818645,6.0,0.137,0.5,507101,d
1,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,1.5,8904223818645,6.0,0.137,0.5,486886,d
2,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,1.5,8904223818645,6.0,0.137,0.5,532484,d
3,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,1.5,8904223818645,6.0,0.137,0.5,143001,b
4,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,1.5,8904223818645,6.0,0.137,0.5,515591,d
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49719,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,0.5,8904223819512,1.0,0.210,0.5,325207,b
49720,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,0.5,8904223819512,1.0,0.210,0.5,303702,b
49721,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,0.5,8904223819512,1.0,0.210,0.5,313301,b
49722,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,0.5,8904223819512,1.0,0.210,0.5,173212,e


In [25]:
invoice_df = invoice_df.merge(rates_long, on =["Zone", "Type_of_Shipment"])

In [26]:
# Creating and adding new column of Expected charged as per X 
invoice_df["Expected Charge as per X (Rs.)"] = invoice_df["Courier_Charged_slab"] * invoice_df["Rate"]

In [27]:
invoice_df

,AWB Code,Order_ID,Total_Weight_Courier,Warehouse Pincode,Customer Pincode_x,Zone,Type_of_Shipment,Billing Amount (Rs.),Courier_Charged_slab,SKU,Order Qty,Total_Weight_X,Total_Weight_slab,Customer Pincode_y,Zone_X,zone_type,Rate,Shipment,Expected Charge as per X (Rs.)
0,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,1.5,8904223818645,6.0,0.137,0.5,507101,d,fwd_d_fixed,45.4,fwd_,68.1
1,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,1.5,8904223818645,6.0,0.137,0.5,507101,d,fwd_d_additional,44.8,fwd_,67.2
2,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,1.5,8904223818645,6.0,0.137,0.5,486886,d,fwd_d_fixed,45.4,fwd_,68.1
3,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,1.5,8904223818645,6.0,0.137,0.5,486886,d,fwd_d_additional,44.8,fwd_,67.2
4,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,1.5,8904223818645,6.0,0.137,0.5,532484,d,fwd_d_fixed,45.4,fwd_,68.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99443,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,0.5,8904223819512,1.0,0.210,0.5,313301,b,fwd_d_additional,44.8,fwd_,22.4
99444,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,0.5,8904223819512,1.0,0.210,0.5,173212,e,fwd_d_fixed,45.4,fwd_,22.7
99445,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,0.5,8904223819512,1.0,0.210,0.5,173212,e,fwd_d_additional,44.8,fwd_,22.4
99446,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,0.5,8904223819512,1.0,0.210,0.5,302020,b,fwd_d_fixed,45.4,fwd_,22.7


In [28]:
# Difference Between Expected Charges and Billed Charges (Rs.)
invoice_df["Difference"] = invoice_df["Expected Charge as per X (Rs.)"] - invoice_df["Billing Amount (Rs.)"]
invoice_df

,AWB Code,Order_ID,Total_Weight_Courier,Warehouse Pincode,Customer Pincode_x,Zone,Type_of_Shipment,Billing Amount (Rs.),Courier_Charged_slab,SKU,Order Qty,Total_Weight_X,Total_Weight_slab,Customer Pincode_y,Zone_X,zone_type,Rate,Shipment,Expected Charge as per X (Rs.),Difference
0,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,1.5,8904223818645,6.0,0.137,0.5,507101,d,fwd_d_fixed,45.4,fwd_,68.1,-66.9
1,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,1.5,8904223818645,6.0,0.137,0.5,507101,d,fwd_d_additional,44.8,fwd_,67.2,-67.8
2,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,1.5,8904223818645,6.0,0.137,0.5,486886,d,fwd_d_fixed,45.4,fwd_,68.1,-66.9
3,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,1.5,8904223818645,6.0,0.137,0.5,486886,d,fwd_d_additional,44.8,fwd_,67.2,-67.8
4,1091117222124,2001806232,1.3,121003,507101,d,Forward charges,135.0,1.5,8904223818645,6.0,0.137,0.5,532484,d,fwd_d_fixed,45.4,fwd_,68.1,-66.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99443,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,0.5,8904223819512,1.0,0.210,0.5,313301,b,fwd_d_additional,44.8,fwd_,22.4,-23.0
99444,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,0.5,8904223819512,1.0,0.210,0.5,173212,e,fwd_d_fixed,45.4,fwd_,22.7,-22.7
99445,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,0.5,8904223819512,1.0,0.210,0.5,173212,e,fwd_d_additional,44.8,fwd_,22.4,-23.0
99446,1091121846136,2001811305,0.5,121003,302020,d,Forward charges,45.4,0.5,8904223819512,1.0,0.210,0.5,302020,b,fwd_d_fixed,45.4,fwd_,22.7,-22.7


In [29]:
invoice_df.drop("Warehouse Pincode", axis = 1, inplace = True)

In [30]:
invoice_df

,AWB Code,Order_ID,Total_Weight_Courier,Customer Pincode_x,Zone,Type_of_Shipment,Billing Amount (Rs.),Courier_Charged_slab,SKU,Order Qty,Total_Weight_X,Total_Weight_slab,Customer Pincode_y,Zone_X,zone_type,Rate,Shipment,Expected Charge as per X (Rs.),Difference
0,1091117222124,2001806232,1.3,507101,d,Forward charges,135.0,1.5,8904223818645,6.0,0.137,0.5,507101,d,fwd_d_fixed,45.4,fwd_,68.1,-66.9
1,1091117222124,2001806232,1.3,507101,d,Forward charges,135.0,1.5,8904223818645,6.0,0.137,0.5,507101,d,fwd_d_additional,44.8,fwd_,67.2,-67.8
2,1091117222124,2001806232,1.3,507101,d,Forward charges,135.0,1.5,8904223818645,6.0,0.137,0.5,486886,d,fwd_d_fixed,45.4,fwd_,68.1,-66.9
3,1091117222124,2001806232,1.3,507101,d,Forward charges,135.0,1.5,8904223818645,6.0,0.137,0.5,486886,d,fwd_d_additional,44.8,fwd_,67.2,-67.8
4,1091117222124,2001806232,1.3,507101,d,Forward charges,135.0,1.5,8904223818645,6.0,0.137,0.5,532484,d,fwd_d_fixed,45.4,fwd_,68.1,-66.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99443,1091121846136,2001811305,0.5,302020,d,Forward charges,45.4,0.5,8904223819512,1.0,0.210,0.5,313301,b,fwd_d_additional,44.8,fwd_,22.4,-23.0
99444,1091121846136,2001811305,0.5,302020,d,Forward charges,45.4,0.5,8904223819512,1.0,0.210,0.5,173212,e,fwd_d_fixed,45.4,fwd_,22.7,-22.7
99445,1091121846136,2001811305,0.5,302020,d,Forward charges,45.4,0.5,8904223819512,1.0,0.210,0.5,173212,e,fwd_d_additional,44.8,fwd_,22.4,-23.0
99446,1091121846136,2001811305,0.5,302020,d,Forward charges,45.4,0.5,8904223819512,1.0,0.210,0.5,302020,b,fwd_d_fixed,45.4,fwd_,22.7,-22.7


In [31]:
# Select columns for detailed sheet
order_level = invoice_df[[
    "Order_ID", "AWB Code", "Total_Weight_X", "Total_Weight_slab", 
    "Total_Weight_Courier", "Courier_Charged_slab", "Zone_X", 
    "Zone", "Expected Charge as per X (Rs.)", "Billing Amount (Rs.)", "Difference"
]]



In [32]:
order_level

,Order_ID,AWB Code,Total_Weight_X,Total_Weight_slab,Total_Weight_Courier,Courier_Charged_slab,Zone_X,Zone,Expected Charge as per X (Rs.),Billing Amount (Rs.),Difference
0,2001806232,1091117222124,0.137,0.5,1.3,1.5,d,d,68.1,135.0,-66.9
1,2001806232,1091117222124,0.137,0.5,1.3,1.5,d,d,67.2,135.0,-67.8
2,2001806232,1091117222124,0.137,0.5,1.3,1.5,d,d,68.1,135.0,-66.9
3,2001806232,1091117222124,0.137,0.5,1.3,1.5,d,d,67.2,135.0,-67.8
4,2001806232,1091117222124,0.137,0.5,1.3,1.5,d,d,68.1,135.0,-66.9
...,...,...,...,...,...,...,...,...,...,...,...
99443,2001811305,1091121846136,0.210,0.5,0.5,0.5,b,d,22.4,45.4,-23.0
99444,2001811305,1091121846136,0.210,0.5,0.5,0.5,e,d,22.7,45.4,-22.7
99445,2001811305,1091121846136,0.210,0.5,0.5,0.5,e,d,22.4,45.4,-23.0
99446,2001811305,1091121846136,0.210,0.5,0.5,0.5,b,d,22.7,45.4,-22.7


## Creating a summary table 

In [34]:
# Total orders where X has been correctly charged
# Total orders where X has been overcharged
# Total orders where X has been undercharged

In [35]:
# Total orders where X has been correctly charged
correctly_charged = (order_level["Billing Amount (Rs.)"] == order_level["Expected Charge as per X (Rs.)"]).sum()
correctly_charged

0

In [36]:
# Total amount where X has been correctly charged
correct_charge = (order_level["Billing Amount (Rs.)"] == order_level["Expected Charge as per X (Rs.)"])
total_sum = order_level[correct_charge]["Billing Amount (Rs.)"].sum()
total_sum

0.0

In [37]:
# Total orders where X has been overcharged
overcharged = (order_level["Billing Amount (Rs.)"] > order_level["Expected Charge as per X (Rs.)"]).sum()
overcharged

99448

In [38]:
# Total amount where X has been overcharged
over_charged = (order_level["Billing Amount (Rs.)"] > order_level["Expected Charge as per X (Rs.)"])
total_sum1 = order_level[over_charged]["Billing Amount (Rs.)"].sum()
total_sum1

12256780.000000002

In [39]:
# Total orders where X has been undercharged
undercharged = (order_level["Billing Amount (Rs.)"] < order_level["Expected Charge as per X (Rs.)"]).sum()
undercharged

0

In [40]:
#Total amount where X has been undercharged
under_charged = (order_level["Billing Amount (Rs.)"]) < order_level["Expected Charge as per X (Rs.)"]
total_sum2 = order_level[under_charged]["Billing Amount (Rs.)"].sum()
total_sum2

0.0

In [41]:
# Making dictionary
charge_output = {
        "Charge_Status_X" :["Total orders where X has been correctly charged", "Total orders where X has been overcharged",
                            "Total orders where X has been undercharged"],
        "Count_Status_X":[0,99448,0],
        "Amount (Rs.)":[0.0,12256780.000000002,0.0]
}

In [42]:
# Making dataframe from dictionary
summary_df = pd.DataFrame(charge_output)
summary_df

,Charge_Status_X,Count_Status_X,Amount (Rs.)
0,Total orders where X has been correctly charged,0,0.0
1,Total orders where X has been overcharged,99448,12256780.0
2,Total orders where X has been undercharged,0,0.0


In [43]:
# Converting dataframe to csv file
summary_df.to_excel("summary_df.xlsx", index = False)

In [44]:
# Converting df1 into csv file
order_level.to_excel("order_level.xlsx", index = False)